<a href="https://colab.research.google.com/github/yunjinchoidev/ai/blob/main/%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EB%A1%9C_%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.
# 레이블 데이터에 one-hot encoding을 적용합니다.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# 학습을 위한 설정값들을 정의합니다.
learning_rate = 0.001
num_epochs = 30     # 학습횟수
batch_size = 256    # 배치개수
display_step = 1    # 손실함수 출력 주기
input_size = 784    # 28 * 28
hidden1_size = 256
hidden2_size = 256
output_size = 10

In [4]:
# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

def random_normal_intializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [19]:
# tf.keras.Model을 이용해서 ANN 모델을 정의합니다.
class ANN(tf.keras.Model):
  def __init__(self):
    super(ANN, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.output_layer = tf.keras.layers.Dense(output_size,
                                              activation=None,
                                              kernel_initializer=random_normal_intializer_with_stddev_1(),
                                              bias_initializer=random_normal_intializer_with_stddev_1())

  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    logits = self.output_layer(H2_output)

    return logits

# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(learning_rate)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# 모델의 정확도를 출력하는 함수를 정의합니다.
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

# ANN 모델을 선언합니다.
ANN_model = ANN()

# 지정된 횟수만큼 최적화를 수행합니다.
for epoch in range(num_epochs):
  average_loss = 0.
  total_batch = int(x_train.shape[0] / batch_size)
  # 모든 배치들에 대해서 최적화를 수행합니다.
  for batch_x, batch_y in train_data:
    # 옵티마이저를 실행해서 파라마터들을 업데이트합니다.
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    # 평균 손실을 측정합니다.
    average_loss += current_loss / total_batch
  # 지정된 epoch마다 학습결과를 출력합니다.
  if epoch % display_step == 0:
    print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))

# 테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test)) # 정확도: 약 94%

반복(Epoch): 1, 손실 함수(Loss): 251.977615
반복(Epoch): 2, 손실 함수(Loss): 61.022778
반복(Epoch): 3, 손실 함수(Loss): 40.522049
반복(Epoch): 4, 손실 함수(Loss): 30.044832
반복(Epoch): 5, 손실 함수(Loss): 23.601955
반복(Epoch): 6, 손실 함수(Loss): 18.775780
반복(Epoch): 7, 손실 함수(Loss): 15.456141
반복(Epoch): 8, 손실 함수(Loss): 12.510220
반복(Epoch): 9, 손실 함수(Loss): 10.302138
반복(Epoch): 10, 손실 함수(Loss): 8.425043
반복(Epoch): 11, 손실 함수(Loss): 6.996170
반복(Epoch): 12, 손실 함수(Loss): 5.705447
반복(Epoch): 13, 손실 함수(Loss): 4.708868
반복(Epoch): 14, 손실 함수(Loss): 3.852503
반복(Epoch): 15, 손실 함수(Loss): 3.159916
반복(Epoch): 16, 손실 함수(Loss): 2.604775
반복(Epoch): 17, 손실 함수(Loss): 2.124114
반복(Epoch): 18, 손실 함수(Loss): 1.651320
반복(Epoch): 19, 손실 함수(Loss): 1.335045
반복(Epoch): 20, 손실 함수(Loss): 1.093363
반복(Epoch): 21, 손실 함수(Loss): 0.856428
반복(Epoch): 22, 손실 함수(Loss): 0.700567
반복(Epoch): 23, 손실 함수(Loss): 0.624265
반복(Epoch): 24, 손실 함수(Loss): 0.444824
반복(Epoch): 25, 손실 함수(Loss): 0.312891
반복(Epoch): 26, 손실 함수(Loss): 0.272124
반복(Epoch): 27, 손실 함수(Loss): 0.248476
